# "데이터 진흥원 파일 자동 다운로드"
### 작성자: 김학건
### 날짜: 2019-01-03
- .gitIgnore: PASSWORD.json
- 성능보다 오류 감소를 위해 코드를 짰음, 그래서 time.sleep()이 많음
    - 추후 수정 예정
- 먼저, 작동하도록 짠 코드라 더러움


<img src="chromeSetting.PNG" alt="Drawing" style="width: 400px;"/>

# 라이브러리

In [ ]:
from selenium import webdriver
import pandas as pd
import time, os, re, json 

# 함수 정의

In [10]:
# 함수: 프레임 바꾸기
def fnSwitchToMain() :
#     print("* switchToMain")
    driver.switch_to_default_content()
    driver.switch_to.frame('main')

def fnSwitchToFrame(FRAME) :
#     print('* switchToMain_' + FRAME)
    driver.switch_to_default_content()
    driver.switch_to.frame('main')
    driver.switch_to.frame(FRAME)
    
# 함수 로그인
def fnLogin(ID, PW):
    fnSwitchToMain()
    temp_element = driver.find_element_by_name("USERS_ID")
    temp_element.clear()
    temp_element.send_keys(ID)
    
    temp_element = driver.find_element_by_name("USERS_PASSWD")
    temp_element.clear()
    temp_element.send_keys(PW)

# 버튼 누르기
def fnBtnClick(FRAME, NAME):
    fnSwitchToFrame(FRAME)
    driver.find_element_by_class_name(NAME).click()

# 검색어 입력
def fnInputKeyword(NAME, KEYWORD):
    fnSwitchToFrame('mainPanel')
    temp_element = driver.find_element_by_name(NAME)
    temp_element.clear()
    temp_element.send_keys(KEYWORD)
    temp_element.send_keys(u'\ue007') # enter
    
def fnDownloadFile(title, sender, date):  
    try:
        time.sleep(1)
        print("[fn: 다운로드 시작]")
        index = 2
        count = 0

        fnSwitchToFrame('mainPanel')

        past = time.time()
#         time.sleep(1)
        #download
        download_count = len(driver.find_elements_by_class_name("k_txAliL"))


        print("\t * " + str(download_count) + "개의 파일을 다운로드 하겠습니다.")
        while  count < download_count:
            xpath = '//*[@id="mail_box_layout"]/table[1]/tbody/tr/td/table/tbody/tr/td/form/table/tbody/tr[' + str(index) + ']/td[2]/a'
            download_click = driver.find_elements_by_xpath(xpath)

            file_name = download_click[0].text
            print("  \t[다운로드 (" + str(count + 1) + ")] " + file_name)
            download_click[0].click()

            time.sleep(1)
            # 엑셀 저장용
            download_list.append(file_name)
            title_list.append(title)
            sender_list.append(sender)
            date_list.append(date)
            # 엑셀 저장용

            index += 1
            count += 1

#         time.sleep(1)
        present = time.time()
        return [past, present]

    except Exception as e:
        print("\n[오류] fnDownloadFile()에서 오류가 났습니다. 다시 실행해주세요")
        print(e)

        
def fnChangeFileName(time_past_present, sender, date):
    print("\t[fn: 파일명 변경]", end="")
    try: 
#         time.sleep(1)
        past = time_past_present[0]
        present = time_past_present[1]

        file_index = 1

        for (dirpath, dirnames, filenames) in os.walk(PATH):
            for filename in filenames:
                f = '/'.join([dirpath,filename])
                mtime = os.path.getmtime(f)
                if(past < mtime and mtime < present): 
                    new_file_name = "[파일명]" + sender + "(" + str(file_index) + ")"+ "_" + date[:10] + "_" + filename
                    new_file_name_regexp = re.sub('[\/:*?"<>|]','',new_file_name)
                    os.rename(f, '/'.join([dirpath, new_file_name_regexp]))
                    file_index += 1
        print("\t*완료*")
                    
    except Exception as e:
        print("\n[오류] fnChangeFileName()에서 오류가 났습니다. 다시 실행해주세요")
        print(e)
        
def fnTitleClickPerGroup():
    try:
        title_click = driver.find_elements_by_xpath('//*[contains(@id, "mailTitleStr")]')
        index = 0
        while index < (len(title_click)):
            time.sleep(1)
            # 제목 객체는 이상하게 계속 만들어줘야함
            title_click = driver.find_elements_by_xpath('//*[contains(@id, "mailTitleStr")]')

            # 페이지 이동
            title_click[index].click()

            # 파일 정보
            title = driver.find_elements_by_xpath('//*[@id="mail_box_layout"]/form/table[3]/tbody/tr[1]/td[2]/table/tbody/tr/td[1]')[0].text
            sender = driver.find_elements_by_xpath('//*[@id="mail_box_layout"]/form/table[3]/tbody/tr[3]/td[2]/table/tbody/tr/td[1]/a')[0].text
            date = driver.find_elements_by_class_name('t_left10')[1].text

            print("====================")
            print("  [순번]: " + str(index + 1) + "/" + str(len(title_click)))
            print("  [제목]: " + title)
            print("  [제목]: " + sender)
            print("  [날짜]: " + date)
            print("====================")

            time_past_present = fnDownloadFile(title, sender, date) # 파일 다운로드 진행
            fnDownloadCheck() # 파일 다운로드 완료
            fnChangeFileName(time_past_present, sender, date)# 파일 명 변경

            # 전 페이지로 돌아가기
            driver.execute_script("window.history.go(-1)")
            index += 1

        print("\n\n [완료] 파일 다운로드가 모두 완료 되었습니다. (총 " + str(index) + "개)")

    except Exception as e:
        print("\n[오류] fnTitleClickPerGroup()에서 오류가 났습니다. 다시 실행해주세요")
        print(e)
        driver.execute_script("window.history.go(-1)")



# 변수 설정

In [ ]:
# 변수 설정

PATH = os.path.expanduser('~/Downloads')

# JSON 에서 정보 가져오기(보안)
with open('PASSWORD.json') as f:
    loginData = json.load(f)

WEB_URL = loginData["web_url" ]
ID = loginData["id"]
PW = loginData["pw"]

title_list = []
sender_list = []
date_list = []
download_list =[]

# 실행

- 1. 페이지 열기 + 로그인

In [4]:
# 창열기
driver = webdriver.Chrome('./driver/chromedriver')
driver.get(WEB_URL)
driver.maximize_window()

fnLogin(ID, PW)
driver.find_element_by_class_name('loginBtn').click()

C:\Users\deet1\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use driver.switch_to.default_content instead
  after removing the cwd from sys.path.


- 2. 검색어 입력

In [5]:
# 클릭: 받은 편지함
fnBtnClick('api', "receiveMail")

# 입력: 메일함 검색
INPUT = input("[검색어 입력] = ")
fnInputKeyword("S_search_strKeyword", INPUT)

C:\Users\deet1\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use driver.switch_to.default_content instead
  if __name__ == '__main__':


[검색어 입력] = 2018중소중견기업 디자인인력지원사업 사업장가입자명부 등 제출 서류 안내


In [20]:
# 여기서 오류

def fnDownloadCheck():
    try:
        print("\t[fn: 다운로드 여부 확인]", end="")
        first_boolean = True
        for file in os.listdir(PATH):
            if(first_boolean):
                print(first_boolean)
                first_boolean = False        
                print(" \t\t 다운로드 중 = " + file +" ")
            
            if ".crdownload" in file:
                print("*", end="")
                time.sleep(0.5) #0.5초뒤 다시 확인
                fnDownloadCheck() # 재귀
        
        print("\t*완료*")
    except Exception as e:
        print("\n[오류] fnDownloadCheck()에서 오류가 났습니다. 다시 실행해주세요")
        print(e)

- 3. 파일 다운로드

In [24]:
fnSwitchToFrame('mainPanel')

total_page = 1
elems = driver.find_elements_by_xpath("//a[@href]")

last_href = elems[-1].get_attribute("href")
if "goMailListRefresh" in last_href:
    total_page = re.findall(r'\d+', last_href)[0]
print("[총 페이지 수] " + str(total_page))
print("====================")

i = 1
while i < int(total_page) + 1:
    try:
        print("  [페이지]: " + str(i) + "/" + str(total_page))
        fnTitleClickPerGroup()
        if i != 1:
            elem = driver.find_element_by_css_selector("a[href*='goMailListRefresh(" + str(i + 1) + ")']")
            elem.click()
        print("페이지 끝 = " + str(i))
        i += 1
    except Exception as e:
        print('pass= ' + str(e))
        print(e)

C:\Users\deet1\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use driver.switch_to.default_content instead
  if __name__ == '__main__':


[총 페이지 수] 8
  [페이지]: 1/8
  [순번]: 1/30
  [제목]: 넵플러스 재송부 드립니다. FW: RE: 2018중소중견기업 디자인인력지원사업 사업장가입자명부 등 제출 서류 안내
  [제목]: 임샘 <ssam@nep-plus.com>
  [날짜]: 2019-01-10 09:43:43
[fn: 다운로드 시작]
	 * 4개의 파일을 다운로드 하겠습니다.
  	[다운로드 (1)] 2. 12월 최완석 급여이체증.jpg
  	[다운로드 (2)] 1. 최완석 12월 급여명세서.jpg
  	[다운로드 (3)] 첨무 1. 2018 디자인인력지원사업 성과입력 (넵플러스).pptx
  	[다운로드 (4)] 넵플러스 사업장 가입자명부 (190102).pdf
	[fn: 다운로드 여부 확인]True
 		 다운로드 중 = 1. 최완석 12월 급여명세서.jpg 
	*완료*
	[fn: 파일명 변경]	*완료*
  [순번]: 2/30
  [제목]: RE: 2018중소중견기업 디자인인력지원사업 사업장가입자명부 등 제출 서류 안내
  [제목]: 최정미 <vine0857@naver.com>
  [날짜]: 2019-01-08 17:30:51
[fn: 다운로드 시작]
	 * 1개의 파일을 다운로드 하겠습니다.
  	[다운로드 (1)] 사회보험가입자명.pdf
	[fn: 다운로드 여부 확인]True
 		 다운로드 중 = desktop.ini 
	*완료*
	[fn: 파일명 변경]	*완료*

[오류] fnTitleClickPerGroup()에서 오류가 났습니다. 다시 실행해주세요
list index out of range
페이지 끝 = 1
  [페이지]: 2/8


 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: c


Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

  [페이지]: 2/8


 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

  [페이지]: 2/8


 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable

  [페이지]: 2/8


 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

  [페이지]: 2/8


 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable 



 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

  [페이지]: 2/8


 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable to locate ele

pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

  [페이지]: 2/8


 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable to locate element: {"method":"css selector","selec



 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

  [페이지]: 2/8


 [완료] 파일 다운로드가 모두 완료 되었습니다. (총 0개)
pass= Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[href*='goMailListRefresh(3)']"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)

Message: no such element: Unable to locate ele

KeyboardInterrupt: 

- 6. 엑셀 파일로 저장

In [ ]:
# 엑셀 파일 저장
from pandas import ExcelWriter
print("[엑셀] 파일을 저장합니다.")

download_list_df = pd.DataFrame.from_dict(
        {
        'download_list':download_list,
        "title_list" : title_list,
        "sender_list" : sender_list,
        "date_list" : date_list,
            
        }
    )

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime() );
new_file_name_regexp = re.sub('[\/:*?"<>|]','', "[다운로드]" + INPUT + "_" + str(current_time) )

writer = ExcelWriter('/'.join([PATH, new_file_name_regexp + '.xlsx']))
download_list_df.to_excel(writer,'다운로드 파일')

print(new_file_name_regexp + '.xlsx ', end = '')
writer.save()

In [ ]:
xpath = '//*[@id="mail_box_layout"]/form/table[5]/tbody/tr/td'
download_click = driver.find_elements_by_xpath(xpath)

print(download_click)

print(download_click[0].text)

In [ ]:
elems = driver.find_elements_by_xpath("//a[@href]")
text = elems[-1].get_attribute("href")
total_page= re.findall(r'\d+', text)[0]

In [ ]:
i = 1
while i < int(total_page):
    try:
        print(i)
        elem = driver.find_element_by_css_selector("a[href*='goMailListRefresh(" + str(i + 1) + ")']");
        elem.click()
        i += 1
    except Exception as e:
        print('pass= ' + str(e))
        print(e)
        continue